# Covid-19 Interactive Statistics


## What is covid-19 ?
>Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic. Common symptoms include fever, cough and shortness of breath. Other symptoms may include fatigue, muscle pain, diarrhoea, sore throat, loss of smell and abdominal pain.The time from exposure to onset of symptoms is typically around five days, but may range from two to fourteen days. While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.

![Coronavirus particle Image](https://www.general-anzeiger-bonn.de/imgs/93/7/8/8/7/6/9/5/7/tok_bb76b55fd9b653b923b95cff64fa6708/w1900_h1071_x1500_y846_5FA082000ED1236B-639385f427dc3327.jpg)

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [2]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
# death_df.head()


In [4]:
#confirmed_df.head()

In [5]:
# recovered_df.head()

In [6]:
# country_df.head()

In [7]:
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

confirmed_df = confirmed_df.rename(columns = {'province/state' : 'state', 'country/region' : 'country'} )
death_df = death_df.rename(columns = {'province/state' : 'state', 'country/region' : 'country'} )
recovered_df = recovered_df.rename(columns = {'province/state' : 'state', 'country/region' : 'country'} )
country_df = country_df.rename(columns = { 'country_region' : 'country'} )


# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())


In [8]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: blue; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

In [9]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)
country_df = country_df.iloc[:, :-1]


# COVID-19 Confirmed/Death/Recovered cases by countries

## Enter the number of countires

In [10]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: blue'
    g = 'background-color: green'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# Slide to check worst hit countires

In [11]:
# # plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [12]:
import plotly.graph_objects as go

# Check the statistics of any country or the World

In [13]:
def plot_cases_for_countries(country):
    labels = ['confirmed', 'death']
    colors = ['blue', 'red']
    mode_size = [6,8]
    line_size = [5, 6]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure()
     
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
 
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();
    
interact(plot_cases_for_countries, country='World')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)  


interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

# Global spread

In [14]:
import folium

world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start = 2, max_zoom = 6, min_zoom = 2)

for i in range(len(confirmed_df)):
    folium.Circle(
    location = [confirmed_df.iloc[i]['lat'],confirmed_df.iloc[i]['long']],
    radius = (int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
    fill = True,
    fill_color = 'yellow',
    color = 'green',
    tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)
    
world_map

<style>
.footer {
  left: 0;
  bottom: 0;
  width: 100%;
  background-color: #A9A9A9;
  color: black;
  text-align: center;
}
</style>

<div class="footer">
    
  <h5 >This is a simple covid-19 live tracker | Built by &copy Shreyas K S, 2020</h5>
    <h5 >Email: <a style="color:#000000;" href="shreyas4156@gmail.com"> shreyas4156@gmail.com</a></h5>
        <h5 >Check out my <a style="color:#000000;" href="https://github.com/shreyas4156"> Github</a></h5>
    <p></p>
        
  
  
   
</div>

,country,last_update,lat,long_,confirmed,deaths,recovered
0,Australia,2020-04-11 18:06:22,-25.000000,133.000000,6303,57,1806
1,Austria,2020-04-11 17:58:30,47.516200,14.550100,13798,337,6604
2,Canada,2020-04-11 18:06:04,60.001000,-95.001000,23195,649,6531
3,China,2020-04-11 12:53:33,30.592800,114.305500,83014,3343,77877
4,Denmark,2020-04-11 17:58:30,56.000000,10.000000,6191,260,2111
...,...,...,...,...,...,...,...
180,West Bank and Gaza,2020-04-11 17:58:30,31.952200,35.233200,268,2,57
181,Western Sahara,2020-04-11 17:58:30,24.215500,-12.885800,4,0,0
182,Yemen,2020-04-11 17:58:30,15.552727,48.516388,1,0,0
183,Zambia,2020-04-11 17:58:30,-13.133897,27.849332,40,2,28
